In [12]:
from web3 import Web3
import json
from eth_account import Account

In [2]:
f = open("blackjack_abi.json")
abi = json.load(f)
f.close()

f = open("blackjack_bytecode.json")
bytecode_json = json.load(f)
f.close()
bytecode = bytecode_json["object"]

In [3]:
ganache_server = "http://127.0.0.1:8545"
web3 = Web3(Web3.HTTPProvider(ganache_server))
web3.isConnected()

True

In [5]:
dealer = web3.eth.accounts[1]
player = web3.eth.accounts[2]
web3.eth.default_account = dealer
web3.eth.default_account

'0xD4012bA179d69DBB8EB7D3Aa83e3010A76AF52ac'

In [27]:
account = Account.from_key("d5046127ca371f85b9268b4c3b6a2b5fa891c66e38c2532726215a7ce4673d32").address
account


'0xD4012bA179d69DBB8EB7D3Aa83e3010A76AF52ac'

In [21]:
#current default is dealer
blackjack_contract = web3.eth.contract(abi=abi, bytecode=bytecode)
tx_hash_construct = blackjack_contract.constructor(player, 100).transact({"value":250})
tx_receipt_construct = web3.eth.waitForTransactionReceipt(tx_hash_construct)
current_game_contract = web3.eth.contract(address=tx_receipt_construct.contractAddress, abi=abi)
print(current_game_contract)

In [22]:
# change account to player
web3.eth.default_account = player
tx_hash_ante = current_game_contract.functions.ante().transact({"value":100})
tx_receipt_ante = web3.eth.waitForTransactionReceipt(tx_hash_ante)
player_hand = current_game_contract.functions.getPlayerHand().call()
player_hand_value = current_game_contract.functions.getPlayerHandValue().call()

In [10]:
player_hand

[44, 21]

In [13]:
player_hand_value

13

In [16]:
def card_num_to_string(number):
    card_str = ""
    mod = number % 13
    div = number // 13

    if mod == 0:
        card_str += "A"
    elif mod == 10:
        card_str += "J"
    elif mod == 11:
        card_str += "Q"
    elif mod == 12:
        card_str += "K"
    else:
        card_str += f"{mod+1}"

    if div == 0:
        card_str += " Spades"
    elif div == 1:
        card_str += " Hearts"
    elif div == 2:
        card_str += " Diamonds"
    elif div == 3:
        card_str += " Clubs"

    return card_str



In [17]:
for num in player_hand:
    print(card_num_to_string(num))

Q Clubs
3 Hearts


In [19]:
tx_hash_ante = current_game_contract.functions.hit().transact()
tx_receipt_ante = web3.eth.waitForTransactionReceipt(tx_hash_ante)
player_hand = current_game_contract.functions.getPlayerHand().call()
player_hand_value = current_game_contract.functions.getPlayerHandValue().call()

In [20]:
for num in player_hand:
    print(card_num_to_string(num))

Q Clubs
3 Hearts
5 Clubs


In [21]:
player_hand_value

18

In [22]:
tx_hash_stand = current_game_contract.functions.hit().transact()
tx_receipt_stand = web3.eth.waitForTransactionReceipt(tx_hash_stand)
player_hand = current_game_contract.functions.getPlayerHand().call()
player_hand_value = current_game_contract.functions.getPlayerHandValue().call()

In [23]:
for num in player_hand:
    print(card_num_to_string(num))
print(player_hand_value)

Q Clubs
3 Hearts
5 Clubs
10 Diamonds
28


In [24]:
current_game_contract.functions.getDealerHand().call()

[37, 44]